In [619]:
# import Panda dependency
import pandas as pd

In [620]:
# filepath as strings, read as DFs
sch_file = "resources\schools_complete.csv"
stu_file = "resources\students_complete.csv"
sch_df = pd.read_csv(sch_file)
stu_df = pd.read_csv(stu_file)

sch_df

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [621]:
# clean student names using list, looping through, and using "replace" to change target string into desired value ("") -- blank in this case
presuffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for psfix in presuffixes:
    stu_df["student_name"] = stu_df["student_name"].str.replace(psfix, "")

C:\Users\nabil\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [622]:
# merge both dfs for complete data set get basic metrics
all_data = pd.merge(stu_df, sch_df, on=["school_name", "school_name"])

student_count = all_data["Student ID"].count()
school_count = len(all_data["school_name"].unique())
total_budget = sch_df["budget"].sum()
mean_read = all_data["reading_score"].mean()
mean_math = all_data["math_score"].mean()

#pass or fail into new DFs, do counts, and percentages
pass_math = all_data[all_data["math_score"] > 69]
pass_read = all_data[all_data["reading_score"] > 69]
pass_both = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]

math_pass_count = pass_math["student_name"].count()
read_pass_count = pass_read["student_name"].count()
pass_both_count = pass_both["student_name"].count()

math_perc = math_pass_count / float(student_count) * 100
read_perc = read_pass_count / float(student_count) * 100
both_perc = pass_both_count / float(student_count) * 100

all_data

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [623]:
#REQUIREMENT #1

#create district summary df
dist_sum_df = pd.DataFrame(
    [{"Total Schools" : school_count,
    "Total Students" : student_count,
    "Total Budget" : total_budget,
    "Ave Math Score" : mean_math,
    "Ave Read Score" : mean_read,
    "% Pass Math" : math_perc,
    "% Pass Read" : read_perc,
    "% Overall Pass" : both_perc}])

#format each column appropriately
dist_sum_df["Total Students"] = dist_sum_df["Total Students"].map("{:,}".format)
dist_sum_df["Ave Math Score"] = dist_sum_df["Ave Math Score"].map("{:.1f}".format)
dist_sum_df["Ave Read Score"] = dist_sum_df["Ave Read Score"].map("{:.1f}".format)
dist_sum_df["% Pass Math"] = dist_sum_df["% Pass Math"].map("{:.0f}".format)
dist_sum_df["% Pass Read"] = dist_sum_df["% Pass Read"].map("{:.0f}".format)
dist_sum_df["% Overall Pass"] = dist_sum_df["% Overall Pass"].map("{:.0f}".format)
dist_sum_df["Total Budget"] = dist_sum_df["Total Budget"].map("${:,.2f}".format)

#reorder columns appropriately
col_order = ["Total Schools","Total Students", "Total Budget", "Ave Math Score", 
             "Ave Read Score", "% Pass Math", "% Pass Read", "% Overall Pass"]
dist_sum_df = dist_sum_df[col_order]

#print output
dist_sum_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65


In [649]:
#REQUIREMENT 2

#determine school types
per_school_types = sch_df.set_index(["school_name"])["type"]
per_school_types

#use value_counts for categorical counting
stud_per_school = stu_df["school_name"].value_counts()

#create series for basic budget table and do calculation
budget_per_school = school_types = sch_df.set_index(["school_name"])["budget"]
per_school_capita = budget_per_school/stud_per_school

#grab mean math and read per school
school_math_mean = all_data.groupby(["school_name"]).mean()["math_score"]
school_read_mean = all_data.groupby(["school_name"]).mean()["reading_score"]

#per school pass math/read create DF and count per DF 
school_pass_math = all_data[(all_data["math_score"] > 69)]
school_pass_read = all_data[(all_data["reading_score"] > 69)]
school_pass_overall = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]
school_pass_m_count = school_pass_math.groupby(["school_name"]).count()["student_name"]
school_pass_r_count = school_pass_read.groupby(["school_name"]).count()["student_name"]
school_pass_all_count = school_pass_overall.groupby(["school_name"]).count()["student_name"]

#calculate percent pass per class per school
math_pass_perc_school = school_pass_m_count / stud_per_school * 100
read_pass_perc_school = school_pass_r_count / stud_per_school * 100
overall_pass_perc_school = school_pass_all_count / stud_per_school * 100

#create  and format output DF
school_sum_df = pd.DataFrame({
    "School Type" : per_school_types,
    "Total Students" : stud_per_school,
    "Total School Budget" : budget_per_school,
    "Per Student Budget" : per_school_capita,
    "Ave Math Score" : school_math_mean,
    "Ave Read Score" : school_read_mean,
    "% Pass Math" : math_pass_perc_school,
    "% Pass Read" : read_pass_perc_school,
    "% Overall Pass" : overall_pass_perc_school})


#FORMAT
# school_sum_df["Total Students"] = school_sum_df["Total Students"].map("{:,}".format)
# school_sum_df["Ave Math Score"] = school_sum_df["Ave Math Score"].map("{:.1f}".format)
# school_sum_df["Ave Read Score"] = school_sum_df["Ave Read Score"].map("{:.1f}".format)
# school_sum_df["% Pass Math"] = school_sum_df["% Pass Math"].map("{:.2f}".format)
# school_sum_df["% Pass Read"] = school_sum_df["% Pass Read"].map("{:.2f}".format)
# school_sum_df["% Overall Pass"] = school_sum_df["% Overall Pass"].map("{:.2f}".format)
# school_sum_df["Total School Budget"] = school_sum_df["Total School Budget"].map("${:,.2f}".format)
# school_sum_df["Per Student Budget"] = school_sum_df["Per Student Budget"].map("${:,.2f}".format)

#print output
school_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [625]:
#REQUIREMENT 3.1 Top 5 schools per overall passing

top_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = False)

top_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94.13,97.04,91.33
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.4,83.8,93.27,97.31,90.95
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.39,97.14,90.60
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.3,84.0,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.59,95.95,90.54


In [638]:
#REQUIREMENT 3.2 Bottom 5 schools per overall passing

top_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = True)

top_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.8,80.7,66.37,80.22,52.99
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,65.99,80.74,53.20
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.68,81.32,53.51
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.3,80.9,66.75,80.86,53.53
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.1,81.0,66.06,81.22,53.54


In [627]:
#REQUIREMENT 3.3A Per School, per Grade, Math Ave Scores

grade9 = all_data[(all_data["grade"]== "9th")]
grade10 = all_data[(all_data["grade"] == "10th" )]
grade11 = all_data[(all_data["grade"] == "11th" )]
grade12 = all_data[(all_data["grade"] == "12th" )]

grade9_math = grade9.groupby(["school_name"]).mean()["math_score"]
grade10_math = grade10.groupby(["school_name"]).mean()["math_score"]
grade11_math = grade11.groupby(["school_name"]).mean()["math_score"]
grade12_math = grade12.groupby(["school_name"]).mean()["math_score"]

grade9_read = grade9.groupby(["school_name"]).mean()["reading_score"]
grade10_read = grade10.groupby(["school_name"]).mean()["reading_score"]
grade11_read = grade11.groupby(["school_name"]).mean()["reading_score"]
grade12_read = grade12.groupby(["school_name"]).mean()["reading_score"]

#create and format math per grade per df
math_per_grade = pd.DataFrame ({
    "9th" : grade9_math,
    "10th" : grade10_math,
    "11th" : grade11_math,
    "12th" : grade12_math})

math_per_grade["9th"] = math_per_grade["9th"].map("{:.1f}".format)
math_per_grade["10th"] = math_per_grade["10th"].map("{:.1f}".format)
math_per_grade["11th"] = math_per_grade["11th"].map("{:.1f}".format)
math_per_grade["12th"] = math_per_grade["12th"].map("{:.1f}".format)

#create and format math per grade per school df
read_per_grade = pd.DataFrame ({
    "9th" : grade9_read,
    "10th" : grade10_read,
    "11th" : grade11_read,
    "12th" : grade12_read})

read_per_grade["9th"] = read_per_grade["9th"].map("{:.1f}".format)
read_per_grade["10th"] = read_per_grade["10th"].map("{:.1f}".format)
read_per_grade["11th"] = read_per_grade["11th"].map("{:.1f}".format)
read_per_grade["12th"] = read_per_grade["12th"].map("{:.1f}".format)

# correct order, remove indices if any
math_per_grade = math_per_grade[["9th", "10th", "11th", "12th"]]
read_per_grade = read_per_grade[["9th", "10th", "11th", "12th"]]
math_per_grade.index.name = None
read_per_grade.index.name = None

math_per_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [628]:
#REQUIREMENT 3.3B Per School, per Grade, Read Ave Scores

read_per_grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


In [660]:
#### Per Spending Ranges, raw and ave math score, read score, and overall score

spending_bins = [0,585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
per_school_capita.groupby(pd.cut(per_school_capita, spend_bins)).count()
school_sum_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

#school summary with spend grouping
school_sum_df.head()

spend_math_grade = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["Ave Math Score"]
spend_math_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Math"]
spend_read_grade = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["Ave Read Score"]
spend_read_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Read"]
overall_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Pass"]

#creat output df
spend_sum_df = pd.DataFrame({
          "Ave Math Score" : spend_math_grade,
          "Ave Read Score": spend_read_grade,
          "% Pass Math": spend_math_perc,
          "% Pass Read": spend_read_perc,
          "% Overall Pass": overall_perc})

spend_sum_df["Ave Math Score"] = spend_sum_df["Ave Math Score"].map("{:.1f}".format)
spend_sum_df["Ave Read Score"] = spend_sum_df["Ave Read Score"].map("{:.1f}".format)
spend_sum_df["% Pass Math"] = spend_sum_df["% Pass Math"].map("{:.0f}".format)
spend_sum_df["% Pass Read"] = spend_sum_df["% Pass Read"].map("{:.0f}".format)
spend_sum_df["% Overall Pass"] = spend_sum_df["% Overall Pass"].map("{:.0f}".format)

spend_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,73,84,63
$645-675,77.0,81.0,66,81,54
